# notebook for running summa
based on Create_Run_Files.py

In [1]:
# netcdf/numpy/xray/stats
import numpy as np
from datetime import datetime, timedelta
import pandas as pd
import xarray as xr
from scipy.stats.stats import pearsonr

# OS interaction
import sys, pickle, os

# import plotting
import seaborn as sns
import matplotlib
from matplotlib.pyplot import subplots
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.basemap import Basemap

# Offline Turbulence Package
import turbpy

# Customize
sns.set_style("whitegrid")
sns.set_context('paper')
%matplotlib inline

In [2]:
# --------------------------------------------------------------------------------------------------------------------
# Directory Lists
# Unix
if 'linux' in sys.platform:
    dir_pre = '/home/lapok/gdrive/'
# Mac
elif 'darwin' in sys.platform:
    dir_pre = '/Users/karllapo/gdrive/'

# Project specific directories
dirProj = dir_pre + 'SnowHydrology/proj/ModTsfc/'
dirPrint = dirProj + 'Graphics'
dirData = dirProj + 'data'

# Summa specific directories/names
dirSumma = dirProj + 'summa/'
projName = 'summaTestCases'

# Create_Run_Files.py
## Description
- Sets up multiple paramter/option runs for the FUSE_SNOW model. Each Run creates a new folder under /settings/SITE/R_X
- If only one Run number is specified, then no values are updated (i.e. param_2_vary is ignored)

## Instructions
1) Modify Input Values  below

2) Run ./Create_Run_Files.py

## Input
- Run_IDs: Number values for new created run files (i.e R_46)
- Site_ID_all: Names of all sites to be included, same as site folder names, (Requires Input, Settings, and output folder set up)
- new_param_all: Paramter names to vary, (From /settings/summa_zParamInfo.txt)
- N_param_itr: Number of values to vary each paramter, (divided evenly between min and max, defined in summa_zParamInfo.txt)

# Define Variables/Parameteres Used
- cPr: Index of current Multiparameter run set
- c_new_param: Name of current parameter
- NPruns: Total number of Multiparamter runs
- cSite: Index of current Site
- c_Site_ID: Name of current Site
- NSites: Total number of Sites


In [3]:
# Set up other paths
# Format for calling the docker version of summa (from Bart's shell script)
# run_exe      = "docker run -v " + ${BASEDIR} + ":/" + ${summaTestCases} + " bartnijssen/summa:docker"
run_exe      = "docker run -v " + dirSumma + ":/" + projName + " bartnijssen/summa:docker"

dirSettings = dirSumma + "settings/"
dirInput = dirSumma + "input/"
dirOutput = dirSumma + "output/"

In [4]:
# Arguments that will eventually be passed to this function
nRuns = 1
siteName = 'CDP'
os.chdir(dirData)
dat = xr.open_dataset('CDP.ModTsfc.ModelForcing_wy2006.nc')

In [5]:
# 1) Run IDs
runIDs = nRuns

In [6]:
# 2) Define Sites to Use
Site_ID_all = siteName

In [7]:
# 3) Defind start and stop time (make sure only one date is uncommented!)
dateStart = dat.time[0]
dateEnd = dat.time[-1]

In [8]:
# 4) Define Parameters to modify from default values and Define values for each new_param_all
# SNQ
new_param_all = ['tempCritRain',
                 'tempRangeTimestep',
                 'heightCanopyTop',
                 'heightCanopyBottom',
                 'winterSAI',
                 'summerLAI',
                 'maxMassVegetation',
                 'f_impede',
                 'rootingDepth',
                 'zmax'] #,'theta_sat', 'theta_res',  'vGn_alpha',  'vGn_n','k_soil']
new_param_val = [273.66, 1, 0.05, 0.01, 0.01, 0.5, 1, 0, 0.1, 0.1]
#,          0.401,     0.136,          -0.84,    1.30,   0.0015]


In [9]:
# 5) Define which paramter to allow to vary (between min and max in summa_zLocalParamInfo)
# Note: this overwrites the value given in new_param_val.
param_2_vary  = 'mw_exp'
# Run Info
NPruns = len(Run_IDs)
NSites = len(Site_ID_all)


NameError: name 'Run_IDs' is not defined

In [10]:
# Dictionary of default decisions
allDecisionsDefault = {}
allDecisionsDefault['soilCatTbl'] = 'ROSETTA'  # (03) soil-category dateset
allDecisionsDefault['vegeParTbl'] = 'USGS'  # (04) vegetation category dataset
allDecisionsDefault['soilStress'] = 'NoahType'  # (05) choice of funct. for soil moisture control on stomatal resistance
allDecisionsDefault['stomResist'] = 'BallBerry'  # (06) choice of function for stomatal resistance
allDecisionsDefault['num_method'] = 'itertive'  # (07) choice of numerical method
allDecisionsDefault['fDerivMeth'] = 'analytic'  # (08) method used to calculate flux derivatives
allDecisionsDefault['LAI_method'] = 'specified'  # (09) method used to determine LAI and SAI
allDecisionsDefault['f_Richards'] = 'mixdform'  # (10) form of Richard's equation
allDecisionsDefault['groundwatr'] = 'noXplict'  # (11) choice of groundwater parameterization
allDecisionsDefault['hc_profile'] = 'pow_prof'  # (12) choice of hydraulic conductivity profile
allDecisionsDefault['bcUpprTdyn'] = 'nrg_flux'  # (13) type of upper boundary condition for thermodynamics
allDecisionsDefault['bcLowrTdyn'] = 'zeroFlux'  # (14) type of lower boundary condition for thermodynamics
allDecisionsDefault['bcUpprSoiH'] = 'liq_flux'  # (15) type of upper boundary condition for soil hydrology
allDecisionsDefault['bcLowrSoiH'] = 'drainage'  # (16) type of lower boundary condition for soil hydrology
allDecisionsDefault['veg_traits'] = 'CM_QJRMS1998'  # (17) choice of param. for veg roughness & displacement height
allDecisionsDefault['canopyEmis'] = 'simplExp'  # (18) choice of parameterization for canopy emissivity
allDecisionsDefault['snowIncept'] = 'lightSnow'  # (19) choice of parameterization for snow interception
allDecisionsDefault['windPrfile'] = 'logBelowCanopy'  # (20) choice of wind profile through the canopy
allDecisionsDefault['astability'] = 'standard'  # (21) choice of stability function
allDecisionsDefault['canopySrad'] = 'CLM_2stream'  # (22) choice of canopy shortwave radiation method
allDecisionsDefault['alb_method'] = 'varDecay'  # (23) choice of albedo representation
allDecisionsDefault['compaction'] = 'anderson'  # (24) choice of compaction routine
allDecisionsDefault['snowLayers'] = 'jrdn1991'  # (25) choice of method to combine and sub-divide snow layers
allDecisionsDefault['thCondSnow'] = 'jrdn1991'  # (26) choice of thermal conductivity representation for snow
allDecisionsDefault['thCondSoil'] = 'mixConstit'  # (27) choice of thermal conductivity representation for soil
allDecisionsDefault['spatial_gw'] = 'localColumn'  # (28) choice of method for the spatial representation of groundwater
allDecisionsDefault['subRouting'] = 'timeDlay'  # (29) choice of method for sub-grid routing
allDecisionsDefault['snowDenNew'] = 'pahaut_76'  # (30) choice of method for new snow density

myDecisions = {'astability': 'louisinv'}

In [18]:
import settings.summaFilemager
summaFileManager.Desicions(myDecisions,
                            '/Users/karllapo/gdrive/SnowHydrology/proj/ModTsfc/summa/ModTsfc_summa/settings',
                            'SNQ', '1', datetime(2005, 10, 1), datetime(2006, 9, 30))


['CDP', 'CDP.ModTsfc.Daily.nc', 'CDP.ModTsfc.ModelForcing_wy2006.nc', 'CDP.ModTsfc.nc', 'CREST', 'SNQ', 'SNQ.ModTsfc.Daily.nc', 'SNQ.ModTsfc.nc', 'SWA', 'SWA.ModTsfc.Daily.nc', 'SWA.ModTsfc.nc', 'ScottyCreek']


ImportError: No module named 'settings'

In [14]:
#####################################################################################
# Loop through each Site (Index from zero)
#####################################################################################
cRID  = 0 # Index of current Run_IDs
cSite = 0 # Index of current Site
while (cSite < NSites):
    # Define Site Info
    c_Site_ID = Site_ID_all[cSite]
    print(c_Site_ID)

    # If more than one NPruns, then get value to vary
    if NPruns > 1:
        # Get Vaules of param_2_vary from LocalParamInfo
        Pvals =  Create_new.GetParamVals(param_2_vary, NPruns, dirSettings, c_Site_ID)
        print(Pvals)
        # Find Index specifed parameters (new_param_all)
        Iparam = new_param_all.index(param_2_vary)
    
    #####################################################################################
    # Loop through each Parameter set run (Index from zero)
    #####################################################################################
    cPR = 0
    while (cPR < NPruns):
        #print cPR
        # Define current Run ID
        cRID_char = "R_" + str(Run_IDs[cRID])
    print(cRID_char)

        # If more than one NPruns, Update
    if NPruns > 1:
        # Update new paramter value for param_2_vary in new_param_val
        new_param_val[Iparam] = Pvals[cPR]

    # Define new run paths
    c_output_dir = dirOutput + c_Site_ID + "/" + cRID_char
    c_settings_dir = dirSettings + c_Site_ID + "/" + cRID_char
    run_output = c_output_dir + "/Run_output.txt"
        
    # Make needed directories
    if not os.path.exists(c_output_dir):
        os.makedirs(c_output_dir)

    if not os.path.exists(c_settings_dir):
        # Now create 5 needed files for current Run ID
        # Not sure what number 5 is supposed to be....
        os.makedirs(c_settings_dir)

        # 1) Create the file manager
        Create_new.file_Manager(dirSettings, dirInput, dirOutput, c_Site_ID, cRID_char)
        
        # 2) Create the snow decision file
        Create_new.Desicions(Decisions_ALL, dirSettings, c_Site_ID, cRID_char, datestart, dateend)

        # 3) Edit Parameter settings for current run
        Create_new.ParamTrial(new_param_all, new_param_val, dirSettings, c_Site_ID, cRID_char)

        # 4) Create run output file (overwrites previous)
        if not os.path.exists(run_output):
            ftemp = open(run_output, 'w')
            ftemp.close()  # Simple way to make a file

        # End of current Paramter set Run
        cPR = cPR + 1
        cRID = cRID + 1
    # End of all Parameter set Runs
    cSite = cSite + 1
    
# End of all Sites

SNQ_ALL


KeyboardInterrupt: 

In [15]:
print(cRID)

0
